# Generate ET cell output tables from the proofread table and add putative cell type information

In [1]:
import pandas as pd
import numpy as np
import pcg_skel
import tqdm
from meshparty import meshwork
from caveclient import CAVEclient
import datetime
import os

client = CAVEclient('minnie65_phase3_v1')

pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [2]:
# MAIN

now = client.materialize.get_timestamp()

In [3]:
skeldir = 'skeletons'

In [4]:
# Defining the HVA/VISp line:

xz0 = [237415, 26308]
xz1 = [286783, 8960]

x0 = xz0[0]
x1 = xz1[0]
z0 = xz0[1]
z1 = xz1[1]

def soma_in_hva(pt):
    ptz = pt[2]
    ptx = pt[0]
    x_thresh = x1 + (ptz-z1) * (x0-x1) / (z0-z1)
    return ptx > x_thresh

def classify_soma(pt):
    if np.any(np.isnan(pt)):
        return np.nan
        
    if soma_in_hva(pt):
        return 'hva'
    else:
        return 'v1'

First we want to get a list of outputs from the L5-ET cells, but restricted to their axon

In [5]:
et_table = 'bodor_pt_cells'
#client.materialize.version == 688

et_df = client.materialize.query_table(et_table)

In [6]:
# Build the skeletons
nrns = {}

for _, row in tqdm.tqdm(et_df.iterrows()):

    #print(row)
    if os.path.exists(f"{skeldir}/{row['pt_root_id']}.h5"):
        nrns[row["pt_root_id"]] = meshwork.load_meshwork(f"{skeldir}/{row['pt_root_id']}.h5")
   
    else:
    
        nrns[row["pt_root_id"]] = pcg_skel.coord_space_meshwork(
            row["pt_root_id"],
            client=client,
            root_point=row["pt_position"],
            root_point_resolution=[4, 4, 40],
            collapse_soma=True,
            synapses="all",
            synapse_table=client.info.get_datastack_info().get("synapse_table"),
            timestamp = now,
        )

        nrns[row["pt_root_id"]].save_meshwork(f"{skeldir}/{row['pt_root_id']}.h5")        
        
# Get the axons
for rid, nrn in nrns.items():
    is_axon = meshwork.algorithms.split_axon_by_annotation(
        nrn,
        'pre_syn',
        'post_syn',
        return_quality=False
    )
    nrn.anno.add_annotations('is_axon', is_axon, mask=True)

12it [06:39, 33.29s/it]


Filter for presynaptic outputs on ET cell axons and concatenate into one dataframe:

In [7]:
# MAIN
#Filter for presynaptic outputs on ET cell axons and concatenate into one dataframe, adding synapse distance:

pre_dfs = []
for rid in et_df["pt_root_id"]:
    syn_filt = nrns[rid].anno.pre_syn.filter_query(
            nrns[rid].anno.is_axon.mesh_mask
    )
    df = syn_filt.df
    df['dist_to_root'] = nrns[rid].distance_to_root(syn_filt.mesh_index)
    df['distance_rank'] = df['dist_to_root'].rank()
    df.attrs = {}
    pre_dfs.append(df)

et_pre_df= pd.concat(pre_dfs, ignore_index=True)
et_pre_df['pre_pt_root_id'] = client.chunkedgraph.get_roots(et_pre_df['pre_pt_supervoxel_id'], timestamp=now)
et_pre_df['post_pt_root_id'] = client.chunkedgraph.get_roots(et_pre_df['post_pt_supervoxel_id'], timestamp=now).astype('int')


In [ ]:
# MAIN
#Add nucleus ID to dataframe


et_pre_df = (
    et_pre_df.merge(
        soma_df[["id", "pt_root_id"]].rename(
            columns={"id": "pre_nucleus_id"}
        ),
        left_on="pre_pt_root_id",
        right_on="pt_root_id",
        how="left",
    ).drop(columns="pt_root_id")
)


In [8]:
# MAIN
# Get single soma root ids and add cell types

soma_df = client.materialize.query_table(
    "nucleus_neuron_svm", filter_equal_dict={"cell_type": "neuron"}
)


# Add number of post_synaptic soma on a segment ID
soma_df['count_soma'] = soma_df.groupby('pt_root_id').transform('count')['id']
num_soma_df = soma_df.drop_duplicates(subset='pt_root_id')[['pt_root_id', 'count_soma']].rename(
    columns={"count_soma": "num_soma"})
num_soma_df = num_soma_df.drop_duplicates(subset="pt_root_id", keep='first')


def number_of_soma(row):               
    if pd.isna(row['num_soma']) == True:
          return 0    
    else:
          return row['num_soma']  
num_soma_df['num_soma'] = num_soma_df.apply(number_of_soma, axis=1)


# Remove all duplicates
soma_df = soma_df.drop_duplicates(subset="pt_root_id", keep='first')

# Download all the other tables we want to pull info from
NEURD_df = client.materialize.query_table("baylor_e_i_model_v1").drop_duplicates('pt_root_id', keep=False)

metamodel_df = client.materialize.query_table(
    "aibs_soma_nuc_metamodel_preds_v117",
    filter_equal_dict={"classification_system": "aibs_neuronal"},
).drop_duplicates('pt_root_id', keep=False)

mtypes_model_df = client.materialize.query_table(
    "aibs_soma_nuc_exc_mtype_preds_v117",
    filter_equal_dict={"classification_system": "aibs_coarse_excitatory"},
).drop_duplicates('pt_root_id', keep=False)

# Enrich soma_df with all this info
soma_df = (
    soma_df.merge(
        NEURD_df[["pt_root_id", "cell_type"]].rename(
            columns={"cell_type": "NEURD_class"}
        ),
        on="pt_root_id",
        how="left",
    )
    .merge(
        metamodel_df[["pt_root_id", "cell_type"]].rename(
            columns={"cell_type": "metamodel_cell_type"}
        ),
        on="pt_root_id",
        how="left",
    )
    .merge(
        mtypes_model_df[["pt_root_id", "cell_type"]].rename(
            columns={"cell_type": "mtypes_model_cell_type"}
        ),
        on="pt_root_id",
        how="left",
    )
)


Table Owner Notice on nucleus_neuron_svm: Please cite https://doi.org/10.1101/2022.07.20.499976 when using this table.


In [10]:
# MAIN
#Add class labels to soma_df
def standard_class_metamodel(row):
        
    if row['metamodel_cell_type'] == 'MC':
          return 'inhibitory'
 
    if row['metamodel_cell_type'] == 'BC':
          return 'inhibitory'
          
    if row['metamodel_cell_type'] == 'NGC':
          return 'inhibitory'
        
    if row['metamodel_cell_type'] == 'BPC':
          return 'inhibitory'
     
    if row['metamodel_cell_type'] == 'none':
          return None
               
    if pd.isna(row['metamodel_cell_type']) == True:
          return None
        
    else:
          return 'excitatory' 

soma_df['metamodel_class'] = soma_df.apply(standard_class_metamodel, axis=1)

In [ ]:
#MAIN
#standardize class labels
def standard_class_NEURD(row):
        
    if pd.isna(row['NEURD_class']) == True:
          return None
    else:
          return row['NEURD_class']  

soma_df['NEURD_class'] = soma_df.apply(standard_class_NEURD, axis=1)

In [ ]:
#MAIN
#standardize sub_class labels
def standard_subclass_metamodel(row):
        
    if row['metamodel_cell_type'] == '6P-IT':
          return '6P'

    if row['metamodel_cell_type'] == '6P-CT':
          return '6P'
    
    if pd.isna(row['metamodel_cell_type']) == True:
          return None
    else:
          return row['metamodel_cell_type']  

soma_df['metamodel_cell_type'] = soma_df.apply(standard_subclass_metamodel, axis=1)

In [ ]:
#MAIN
#standardize sub_class labels
def standard_subclass_mytpes_model(row):
        
    if row['mtypes_model_cell_type'] == 'L3c':
          return '23P'
    
    if row['mtypes_model_cell_type'] == 'L5ET':
          return '5P-ET'

    if row['mtypes_model_cell_type'] == 'L2b':
          return '23P'
        
    if row['mtypes_model_cell_type'] == 'L6a':
          return '6P'
  
    if row['mtypes_model_cell_type'] == 'L4c':
          return '4P'
        
    if row['mtypes_model_cell_type'] == 'L6c':
          return '6P'
        
    if row['mtypes_model_cell_type'] == 'L6CT':
          return '6P'
        
    if row['mtypes_model_cell_type'] == 'L6b':
          return '6P'
        
    if row['mtypes_model_cell_type'] == 'L4a':
          return '4P'
        
    if row['mtypes_model_cell_type'] == 'L2a':
          return '23P'
        
    if row['mtypes_model_cell_type'] == 'L3b':
          return '23P'
        
    if row['mtypes_model_cell_type'] == 'L3a':
          return '23P'

    if row['mtypes_model_cell_type'] == 'L5b':
          return '5P-IT'

    if row['mtypes_model_cell_type'] == 'L4b':
          return '4P'

    if row['mtypes_model_cell_type'] == 'L5a':
          return '5P-IT'

    if row['mtypes_model_cell_type'] == 'L5NP':
          return '5P-NP'

    if row['mtypes_model_cell_type'] == 'L6wm':
          return '6P'

    if pd.isna(row['mtypes_model_cell_type']) == True:
          return None
    else:
          return row['mtypes_model_cell_type']  

soma_df['mtypes_model_cell_type'] = soma_df.apply(standard_subclass_mytpes_model, axis=1)


In [11]:
# MAIN
#Merge all this info from cell types into the synapse dataframe, as well as add area locations.

#merge presynaptic nucleous ID
synapse_table = et_pre_df.merge(
    soma_df[
        ["id", "pt_root_id", "pt_position", "NEURD_class", "metamodel_class", "metamodel_cell_type", "mtypes_model_cell_type"]
    ].rename(columns={"pt_position": "post_soma_pt"}).rename(columns={"id": "post_nucleus_id"}),
    left_on="post_pt_root_id",
    right_on="pt_root_id",
    how="left",
).drop(columns="pt_root_id")

synapse_table["post_soma_area"] = synapse_table['post_soma_pt'].apply(classify_soma)

synapse_table = synapse_table.merge(
    et_df[['pt_root_id', 'pt_position']].rename(columns={'pt_position': 'pre_soma_pt'}),
    left_on='pre_pt_root_id',
    right_on='pt_root_id',
    how='left',
).drop(columns='pt_root_id')

synapse_table["pre_soma_area"] = synapse_table['pre_soma_pt'].apply(classify_soma)

synapse_table = synapse_table.rename(columns={"cell_type_pred": "aibs_auto_subclass"})


# load manual labels

manual_multi_df = client.materialize.live_live_query(
    'pt_synapse_targets',
    timestamp="now",
    metadata=False,
)
#manual_multi_df = client.materialize.query_table("pt_synapse_targets")
manual_multi_df = manual_multi_df.rename(columns={"target_id": "synapse_id"})
#manual_multi_df = client.materialize.query_table("pt_synapse_targets").drop_duplicates('post_pt_root_id', keep=False)
#manual_multi_df['post_pt_root_id'] = manual_multi_df.post_pt_root_id.astype('UInt64')


#manual_multi_df = pd.read_feather('manual_pt.feather')

synapse_table = synapse_table.rename(columns={"id": "synapse_id"})


#merge manual labels

synapse_table = (
    synapse_table.merge(
        manual_multi_df[["synapse_id", "classification_system"]].rename(
            columns={"classification_system": "manual_class"}
        ),
        on='synapse_id',
        how="left",
    )
    .merge(
        manual_multi_df[["synapse_id", "cell_type"]].rename(
            columns={"cell_type": "manual_subclass"}
        ),
        on='synapse_id',
        how="left",
    )
    .merge(
        num_soma_df[["pt_root_id", "num_soma"]],
        left_on='post_pt_root_id',
        right_on='pt_root_id',
        how="left",
    ).drop(columns='pt_root_id')
)  

def number_of_soma(row):               
    if pd.isna(row['num_soma']) == True:
          return 0    
    else:
          return row['num_soma']  
synapse_table['num_soma'] = synapse_table.apply(number_of_soma, axis=1)

In [12]:
# MAIN
# Add alternative nucleous points

nucleus_alternative_df = client.materialize.query_table('nucleus_alternative_points')

synapse_table = synapse_table.merge(
    nucleus_alternative_df[
        ["pt_root_id", "id_ref", "pt_position"]
    ].rename(columns={"pt_position": "post_soma_pt2"}),
    left_on="post_pt_root_id",
    right_on="pt_root_id",
    how="left",
).drop(columns="pt_root_id")


def add_nucleus_ID(row):
   
    if pd.isna(row['id_ref']) == True:
          return row['post_nucleus_id']
    
    else:
          return row['id_ref']  

synapse_table['post_nucleus_id'] = synapse_table.apply(add_nucleus_ID, axis=1)



def add_soma_pt(row):
   
    if pd.isna(row['id_ref']) == True:
          return row['post_soma_pt']
    
    else:
          return row['post_soma_pt2']  

synapse_table['post_soma_pt'] = synapse_table.apply(add_soma_pt, axis=1)



def add_num_soma(row):
   
    if pd.isna(row['id_ref']) == True:
          return row['num_soma']
    
    else:
          if row['num_soma'] > 0:
                print(row['num_soma'])
                print('ID:', row['id_ref'], ' has ', row['num_soma'], ' somata' )
                return row['num_soma']
        
          else:
                return 1  

synapse_table['num_soma'] = synapse_table.apply(add_num_soma, axis=1)

synapse_table = synapse_table.drop(['id_ref', 'post_soma_pt2'], axis=1)

184.0
ID: 177257.0  has  184.0  somata
184.0
ID: 323654.0  has  184.0  somata
184.0
ID: 344893.0  has  184.0  somata
184.0
ID: 621029.0  has  184.0  somata
184.0
ID: 174488.0  has  184.0  somata
184.0
ID: 650632.0  has  184.0  somata
1.0
ID: 401788.0  has  1.0  somata
1.0
ID: 304986.0  has  1.0  somata
1.0
ID: 304986.0  has  1.0  somata
1.0
ID: 304986.0  has  1.0  somata
1.0
ID: 304986.0  has  1.0  somata


In [16]:
#MAIN
#standardize class labels from manual
def standard_class_man(row):
        
    if row['manual_class'] == 'none':
          return None        

    if pd.isna(row['manual_class']) == True:
          return None
         
    else:
          return row['manual_class']  

synapse_table['manual_class'] = synapse_table.apply(standard_class_man, axis=1)

synapse_table = synapse_table[(synapse_table['manual_class'] != 'error')]

In [17]:
#MAIN
#standardize sub_class labels
def standard_subclass_man(row):
        
    if row['manual_subclass'] == 'multisoma':
          return None

    if row['manual_subclass'] == 'DTC':
          return 'MC'               
               
    if row['manual_subclass'] == 'none':
          return None

    if row['manual_subclass'] == '5P-PT':
          return '5P-ET'
        
    if row['manual_subclass'] == 'unclear':
          return None
    
    if pd.isna(row['manual_subclass']) == True:
          return None
    else:
          return row['manual_subclass']  

synapse_table['manual_subclass'] = synapse_table.apply(standard_subclass_man, axis=1)

In [18]:
# MAIN
#QC - CHECK IF THERE ARE DFERRENT MANUAL CLASS LABELS ASIGNED TO THE SAME NEURON 

for ii in synapse_table.post_pt_root_id.unique():

    if len(synapse_table[(synapse_table['post_pt_root_id'] == ii) &
                    pd.notna(synapse_table['manual_class'])].manual_class.unique()) > 1:
    #if len(synapse_table[(synapse_table['post_pt_root_id'] == ii) & (synapse_table['num_soma'] < 2) &
    #                pd.notna(synapse_table['manual_class'])].manual_class.unique()) > 1:
        print(ii)    

In [19]:
# MAIN
#QC - CHECK IF THERE ARE DFERRENT MANUAL SUBCLASS LABELS ASIGNED TO THE SAME NEURON 

for ii in synapse_table.post_pt_root_id.unique():

    if len(synapse_table[(synapse_table['post_pt_root_id'] == ii) & (synapse_table['num_soma'] < 2) 
                         & pd.notna(synapse_table['manual_subclass'])].manual_subclass.unique()) > 1:
        print(ii)
        
    if len(synapse_table[(synapse_table['post_pt_root_id'] == ii) & (synapse_table['num_soma'] < 2) 
                         & pd.notna(synapse_table['manual_subclass'])].manual_subclass.unique()) > 1:

        print(ii) 

In [20]:
# MAIN
#TRANSFER MANUAL SUBCLASS LABELS ACROSS SYNAPSES OF THE SAME NEURON

#Create df with subclass labels and only one post_pt_root_id for IDs that are single somas or orphans
manual_subclass_labels = synapse_table[(synapse_table['num_soma'] <= 1) &
                                      pd.notna(synapse_table['manual_subclass'])].drop_duplicates(subset='post_pt_root_id')

manual_subclass_labels = manual_subclass_labels[['post_pt_root_id', 'manual_subclass']] 

#Transfer the subclass labels using the merge function 
#(In an earlier version I created a new table after this point "#synapse_table_transfer")

synapse_table = synapse_table.merge(manual_subclass_labels, left_on='post_pt_root_id',
                                                      right_on='post_pt_root_id', how='left')

#Transfer the subclass labels on multisoma
def subclass_transfer(row):
   
    if pd.isna(row['manual_subclass_y']) == True:
          return row['manual_subclass_x']
   
    else:
          return row['manual_subclass_y']  

synapse_table['manual_subclass_y'] = synapse_table.apply(subclass_transfer, axis=1)

#Rename columns
synapse_table = synapse_table.rename(columns={"manual_subclass_x": "manual_subclass_original",
                                                                "manual_subclass_y": "manual_subclass"})


In [21]:
#MAIN
#TRANSFER Manual CLASS LABELS ACROSS SYNAPSES OF THE SAME NEURON

#Create df with subclass labels and only one post_pt_root_id for IDs that are single somas or orphans
manual_class_labels = synapse_table[(synapse_table['num_soma'] <= 1) &
                                      pd.notna(synapse_table['manual_class'])].drop_duplicates(subset='post_pt_root_id')

manual_class_labels = manual_class_labels[['post_pt_root_id', 'manual_class']] 

#Transfer the subclass labels using the merge function
synapse_table = synapse_table.merge(manual_class_labels, left_on='post_pt_root_id',
                                                      right_on='post_pt_root_id', how='left')

#Transfer multisoma labels
def class_transfer(row):
   
    if pd.isna(row['manual_class_y']) == True:
          return row['manual_class_x']
   
    else:
          return row['manual_class_y']  

synapse_table['manual_class_y'] = synapse_table.apply(class_transfer, axis=1)

#Rename columns
synapse_table = synapse_table.rename(columns={"manual_class_x": "manual_class_original",
                                              "manual_class_y": "manual_class"})


In [22]:
# MAIN
#QC - CHECK IF THE MANUAL CLASS AND SUBCLASS ARE CONSISTENT

#create new column where class is calculated from subclass
def create_class_from_subclass(row):
    if row['manual_subclass'] == '5P-NP':
          return 'excitatory'
    if row['manual_subclass'] == '5P-ET':
          return 'excitatory'
    if row['manual_subclass'] == '5P-IT':
          return 'excitatory'
    if row['manual_subclass'] == '4P':
          return 'excitatory'
    if row['manual_subclass'] == '6P':
          return 'excitatory'
    if row['manual_subclass'] == '23P':
          return 'excitatory'
    if row['manual_subclass'] == 'BC':
          return 'inhibitory'
    if row['manual_subclass'] == 'MC':
          return 'inhibitory'
    if row['manual_subclass'] == 'BPC':
          return 'inhibitory'
    else:
          return row['manual_subclass']

synapse_table['class_from_subclass'] = synapse_table.apply(create_class_from_subclass, axis=1)


def check_class_from_subclass(row):
   
    if row['manual_class'] == row['class_from_subclass']:
          return 'OK'
   
    else:
          return row['manual_subclass']  

synapse_table['check_class_from_subclass'] = synapse_table.apply(check_class_from_subclass, axis=1)

synapse_table.check_class_from_subclass.unique()


array(['OK', None], dtype=object)

In [23]:
# MAIN
#QC - CHECK DISAGREEMENT BETWEEN NEURD and metamodel LABELS

manual_check1 = synapse_table[(synapse_table['NEURD_class'] == 'excitatory') 
              & (synapse_table['metamodel_class'] == 'inhibitory') & pd.isna(synapse_table['manual_class'])].drop_duplicates(subset='post_pt_root_id')#.post_pt_root_id.unique()

print('number of unchecked disagreements where NEURD is "E" and metamodel is "I": ', len(manual_check1))


manual_check2 = synapse_table[(synapse_table['NEURD_class'] == 'inhibitory') 
              & (synapse_table['metamodel_class'] == 'excitatory') & pd.isna(synapse_table['manual_class'])].drop_duplicates(subset='post_pt_root_id')#.post_pt_root_id.unique()

print('number of unchecked disagreements where NEURD is "I" and metamodel is "E": ', len(manual_check2))


number of unchecked disagreements where NEURD is "E" and metamodel is "I":  0
number of unchecked disagreements where NEURD is "I" and metamodel is "E":  0


In [24]:
#MAIN
#QC - CHECK FOR LABELS WITH NO ENTRIES

manual_check = synapse_table[pd.isna(synapse_table['NEURD_class']) 
              & pd.isna(synapse_table['metamodel_class']) & pd.isna(synapse_table['manual_class'])].drop_duplicates(subset='post_pt_root_id')#.post_pt_root_id.unique()

manual_check

,synapse_id,size,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,pre_pt_position,post_pt_position,ctr_pt_position,pre_pt_level2_id,pre_pt_mesh_ind,pre_pt_mesh_ind_filt,dist_to_root,distance_rank,pre_nucleus_id,post_nucleus_id,post_soma_pt,NEURD_class,metamodel_class,metamodel_cell_type,mtypes_model_cell_type,post_soma_area,pre_soma_pt,pre_soma_area,manual_class_original,manual_subclass_original,num_soma,manual_subclass,manual_class,class_from_subclass,check_class_from_subclass
3835,346241627,19816,104166289513629545,864691136009704622,104166289513624730,864691134991397890,"[286702, 213960, 22939]","[286738, 213900, 22943]","[286680, 213910, 22944]",176223883551048383,1642,1642,302106.928238,107.0,527784,NaN,NaN,None,NaN,None,None,NaN,"[310384, 190160, 24525]",hva,None,None,0.0,None,None,None,OK


In [25]:
#MAIN
#INTEGRATE CLASS LABELS BETWEEN MANUAL AND AUTOMATED LABELS

#generate new consensus column
synapse_table['consensus_class'] = synapse_table['manual_class']

#When there isn't manual label add aibs_v2 label

def integrate_class(row):
    if row['consensus_class'] == None:
          return row['metamodel_class']
    
    else:
          return row['consensus_class']  

synapse_table['consensus_class'] = synapse_table.apply(integrate_class, axis=1)  

In [26]:
#MAIN
#INTEGRATE SUBCLASS LABELS BETWEEN MANUAL AND AUTOMATED LABELS

#generate new consensus column
synapse_table['consensus_subclass'] = synapse_table['manual_subclass']

#When there isn't manual label add aibs_v2 label

def integrate_subclass(row):
    if row['consensus_subclass'] == None:
          return row['metamodel_cell_type']
    if row['consensus_subclass'] == 'inhibitory':
          return None      
    
    else:
          return row['consensus_subclass']  

synapse_table['consensus_subclass'] = synapse_table.apply(integrate_subclass, axis=1)  

In [27]:
#MAIN
#QC - CHECK IF THE INTEGRATED CLASS AND SUBCLASS ARE CONSISTENT

#remove previous columns
synapse_table = synapse_table.drop(['class_from_subclass', 'check_class_from_subclass'], axis=1)
#synapse_table = synapse_table.drop(['class_from_subclass'], axis=1)


#create new column where class is calculated from subclass
def create_class_from_subclass(row):
    if row['consensus_subclass'] == '5P-NP':
          return 'excitatory'
    if row['consensus_subclass'] == '5P-ET':
          return 'excitatory'
    if row['consensus_subclass'] == '5P-IT':
          return 'excitatory'
    if row['consensus_subclass'] == '4P':
          return 'excitatory'
    if row['consensus_subclass'] == '6P':
          return 'excitatory'
    if row['consensus_subclass'] == '6P-IT':
          return 'excitatory'
    if row['consensus_subclass'] == '6P-CT':
          return 'excitatory'
    if row['consensus_subclass'] == '23P':
          return 'excitatory'
    if row['consensus_subclass'] == 'BC':
          return 'inhibitory'
    if row['consensus_subclass'] == 'MC':
          return 'inhibitory'
    if row['consensus_subclass'] == 'NGC':
          return 'inhibitory'
    if row['consensus_subclass'] == 'BPC':
          return 'inhibitory'
    if row['consensus_subclass'] == 'inhibitory':
          return 'unknown'        
        
    else:
          return row['consensus_subclass']

synapse_table['class_from_subclass'] = synapse_table.apply(create_class_from_subclass, axis=1)


def check_class_from_subclass(row):
   
    if row['consensus_class'] == row['class_from_subclass']:
          return 'OK'
   
    else:
          return row['consensus_subclass']  

synapse_table['check_class_from_subclass'] = synapse_table.apply(check_class_from_subclass, axis=1)

synapse_table.check_class_from_subclass.unique()


array(['OK', None], dtype=object)

In [28]:
# MAIN
#SAVE AND READ

#remove columns before saving
synapse_table = synapse_table.drop(['class_from_subclass', 'check_class_from_subclass'], axis=1)


#save et_pre_ct_df
synapse_table.reset_index(drop=True).to_feather("ET_extended_synapse_table.feather")

#READ
#et_pre_ct_df = pd.read_feather('ET_Column_syn_df_NC.feather')